# How to exchange variables among living subkernels

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * Magic `%get` get variable from another kernel
  * Magic `%with` execute the cell in another kernel with input and output variables
  

## Data exchange among kernels

A SoS notebook can have multiple live kernels. A SoS notebook can be used as a collection of cells from multiple independent notebooks. However, the real power of SoS Notebook lies in its ability to exchange data among kernels.

### String interpolation (magic `%expand`) <a id="String_Interpolation"></a>

Cell content is by default sent to SoS or subkernels untouched. A `expand` magic interpolate cell content with variables in the SoS kernel as if the content is a Python f-string. For example,

In [17]:
filename = 'somefile'

In [18]:
echo {filename}

{filename}


In [19]:
%expand
echo {filename}

somefile


In [20]:
filename = 'myfile'
print(f'A filename "{filename}" is passed ')

A filename "myfile" is passed 


In [21]:
%expand
filename = 'myfile'
print(f'A filename "{filename}" is passed ')

A filename "somefile" is passed 


In the last example, `{filename}` is expanded by the `%expand` magic so the following statements are sent to Python 3:
```
filename = 'myfile'
print(f'A filename "somefile" is passed ')
```

As you can imagine, you can keep constants such as filenames and parameters  in SoS and use these information to compose scripts in subkernels for execution.

By default, the `expand` magic expands expressions in braces `{ }`. However, as you can see from the last example, `%expand` can be awkward if the script already contains `{ }`. Although the official method is to double the braces, as in

In [22]:
%expand
filename = 'myfile'
print(f'A filename "{{filename}}" is passed ')

A filename "myfile" is passed 


SoS recommend the use of an alternative sigil so that you do not have to change the syntax of the original script

In [23]:
%expand ${ }
filename = 'myfile'
print(f'SoS filename ${filename}, Python 3 filename: "{filename}"')

SoS filename somefile, Python 3 filename: "myfile"


### Explicit data exchange (magic `%get`) <a id="Explicit_data_exchange"></a>

The SoS kernel provides a mechanism to pass variables between SoS and some subkernels using SoS magics.

For example, magic `%get` can get specified SoS variables from the SoS kernel to the subkernel `ir`.

In [37]:
data = [-1, 0, 1, 2, 3]

In [38]:
%get data
data

[1] -1  0  1  2  3

SoS tries its best to find the best-matching data types between SoS and the subkernel and convert the data in the subkernel's native datatypes (e.g. Python's `DataFrame` to R's `data.frame`), so the variables you get will always be in the subkernel's native data types, not a wrapper of a foreign object (for example objects provided by `rpy2`). 

In [39]:
class(data)

[1] "numeric"

Similarly, using magic `%put`, you can put a variable in the subkernel to the sos kernel.

In [40]:
data[ data < 1] = 0
data[ data > 1] = 1

In [41]:
%put data

In [42]:
%use sos
data

[0, 0, 1, 1, 1]

Variables can also be transferred with options `--in` (`-i`) and `--out` (`-o`) of magics `%use` and `%with`. For example, if you would like to add `2` to all elements in `data` but not sure if `pandas` can do that, you can send the dataframe to `R`, add `2` and send it back.

In [43]:
%with R -i data -o data
data <- data + 2

In [44]:
data

[2, 2, 3, 3, 3]

### Implicit data exchange (`sos*` variables)<a id="Implicit_data_exchange"></a>

In addition to the use of magics `%put` and `%get` and parameters `--in` and `--out` of magics `%use` and `%with` to explicitly exchange variables between SoS and subkernels, SoS automatically shares variables with names starting with `sos` among all subkernels.

For example,

In [45]:
%use R
sos_var = 100

In [46]:
%with Python3
sos_var += 100
print("sos_var is changed to {}".format(sos_var))

sos_var is changed to 200


In [47]:
# I am still in R
sos_var

[1] 200

In [48]:
%use sos
sos_var

200

SoS supports an increasing number of languages and provides [an interface to add support for other languages](Language_Module.html). Please refer to chapter [Supported Language](Supported_Languages.html) for details on each supported language. If your language of choice is not yet supported, please considering adding SoS support for your favoriate kernel with a pull request.

In [49]:
# clean up
%use sos
!rm mydata.csv 

rm: mydata.csv: No such file or directory


### Capture output (Magic `%capture`)

Magic `capture` captures output from a cell, optionally parse it in specified format (option `--as`, and save or append the result to a variable in SoS (`--to` or `--append`). The output of the cell (namely the input to magic `%capture`) can be 

* standard output, `%capture stdout` or just `%capture`
* standard error, `%capture stderr`
* plain text, `%capture text` from `text/plain` of message `display_data`
* markdown, `%capture markdown` from `text/markdown` of message `display_data`
* html, `%capture html` from `text/html` of message `display_data`, or
* raw, which simply returns a list of messages.

For example

In [50]:
%capture --to r_var
cat(paste(c(1,2,3)))

1 2 3

captures standard output of the cell to variable `r_var`

In [51]:
r_var

'1 2 3'

In [52]:
%capture text --to r_var
paste(c(1,2,3))

[1] "1" "2" "3"

In [53]:
r_var

'[1] "1" "2" "3"'

The `capture` magic allows passing information from the output of cells back to SoS. If the output text is structured, it can even parse the output in `json`, `csv` (comma separated values), or `tsv` (tab separated values) formats and save them in Python `dict` or `Pandas.DataFrame`.

For example, 

In [54]:
%capture --as csv --to table
print('a,b\n1,2\n3,4')

a,b
1,2
3,4


In [55]:
table

,a,b
0,1,2
1,3,4


This method is especially suitable for kernels without a language module so there is no way to use a `%get` magic to retrieve information from it. For example, a [`sparql` kernel](https://www.w3.org/TR/rdf-sparql-query/) simply executes sparql queries and return results. It does not have a concept of variable so you will have to capture its output to handle it in SoS:

In [58]:
%capture html --to sparql

%endpoint http://dbpedia.org/sparql
%format JSON

SELECT DISTINCT ?property
WHERE {
   ?s ?property ?person .
   ?person rdf:type foaf:Person .
}
LIMIT 3

Endpoint set to: http://dbpedia.org/sparql
Return format: JSON

property
http://www.w3.org/2002/07/owl#differentFrom
http://www.w3.org/2000/01/rdf-schema#seeAlso
http://www.w3.org/2002/07/owl#sameAs


In [63]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(sparql)
soup.find_all('a')

[<a href="http://www.w3.org/2002/07/owl#differentFrom" target="_other">http://www.w3.org/2002/07/owl#differentFrom</a>,
 <a href="http://www.w3.org/2000/01/rdf-schema#seeAlso" target="_other">http://www.w3.org/2000/01/rdf-schema#seeAlso</a>,
 <a href="http://www.w3.org/2002/07/owl#sameAs" target="_other">http://www.w3.org/2002/07/owl#sameAs</a>]

In addition to option `--to`, the `%capture` magic can also append cell output to an existing variable using paramter `--append`. How newly captured data is appended to the existing variable depends on type of existing and new data. Basically, 

1. If the variable does not exist, `--append` is equivalent to `--to`.
2. If the existing variable is a list, the new data is appended to the list.
3. If the new data has the same type with the existing one, the new data will be appended to existing variable. That it to say, strings are appended to existing string, dictionaries are merged to existing dictionaries, and rows of data frames are appended to existing data frame.

For example, we already have a variable `table`

In [64]:
table

,a,b
0,1,2
1,3,4


In [67]:
%capture --as csv -a table
print('a,b\n5,6\n7,8')

a,b
5,6
7,8


In [68]:
table

,a,b
0,1,2
1,3,4
0,5,6
1,7,8


If you would like to collect results from multiple cells, you can create a list and use it to capture them

In [69]:
results = []

In [70]:
%capture -a results
sh:
  echo result from sh

result from sh


In [71]:
%capture -a results
python:
   print('result from python')

result from python


In [72]:
results

['result from sh\n', 'result from python\n']

## Further reading

* 